In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

In [2]:
train_data = pd.read_csv("twitter_train_data.csv")
test_data = pd.read_csv("twitter_test_data.csv")

In [3]:
train_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
test_data.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [5]:
train_data.drop(['tweet_id','airline','airline_sentiment_gold','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis = 1,inplace = True)
test_data.drop(['tweet_id','airline','airline_sentiment_gold','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis = 1,inplace = True)

In [6]:
del train_data['name']
del test_data['name']

In [7]:
train_data.head()

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


In [8]:
test_data.head()

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...


In [9]:
class_names = list(set(train_data['airline_sentiment']))

In [10]:
class_names

['positive', 'negative', 'neutral']

In [11]:
def getValue(str):
    if str == "negative":
        return 0
    elif str == "neutral":
        return 1
    else:
        return 2

In [12]:
train_data['airline_sentiment_new'] = train_data['airline_sentiment'].apply(getValue)
del train_data['airline_sentiment']
train_data.rename(columns = {'airline_sentiment_new':'airline_sentiment'},inplace = True)

In [13]:
train_data.head()

,text,airline_sentiment
0,"@SouthwestAir I am scheduled for the morning, ...",0
1,@SouthwestAir seeing your workers time in and ...,2
2,@united Flew ORD to Miami and back and had gr...,2
3,@SouthwestAir @dultch97 that's horse radish 😤🐴,0
4,@united so our flight into ORD was delayed bec...,0


In [14]:
set(train_data['airline_sentiment'])

{0, 1, 2}

In [15]:
y_train = train_data['airline_sentiment']
del train_data['airline_sentiment']

In [16]:
x_train = train_data
x_test = test_data

In [17]:
x_train.head()

,text
0,"@SouthwestAir I am scheduled for the morning, ..."
1,@SouthwestAir seeing your workers time in and ...
2,@united Flew ORD to Miami and back and had gr...
3,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,@united so our flight into ORD was delayed bec...


In [18]:
x_test.head()

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...


In [19]:
from nltk.corpus import stopwords

In [20]:
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
import string
punctuations = [i for i in string.punctuation]
punctuations

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [22]:
punctuations = punctuations

In [23]:
stop = stop + punctuations

In [24]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
count_vec = CountVectorizer(max_features = 100,stop_words = stop)

In [27]:
a = count_vec.fit_transform(x_train['text'])

In [28]:
count_vec.get_feature_names()

['agent',
 'airline',
 'airport',
 'americanair',
 'amp',
 'another',
 'back',
 'bag',
 'baggage',
 'bags',
 'call',
 'cancelled',
 'change',
 'check',
 'co',
 'could',
 'customer',
 'day',
 'days',
 'delay',
 'delayed',
 'dm',
 'due',
 'email',
 'even',
 'ever',
 'first',
 'flight',
 'flighted',
 'flightled',
 'flights',
 'fly',
 'flying',
 'gate',
 'get',
 'getting',
 'go',
 'going',
 'good',
 'got',
 'great',
 'guys',
 'help',
 'hold',
 'home',
 'hour',
 'hours',
 'hrs',
 'http',
 'jetblue',
 'know',
 'last',
 'late',
 'let',
 'like',
 'lost',
 'love',
 'luggage',
 'make',
 'minutes',
 'much',
 'need',
 'never',
 'new',
 'next',
 'one',
 'people',
 'phone',
 'plane',
 'please',
 'really',
 'seat',
 'see',
 'service',
 'someone',
 'southwestair',
 'still',
 'take',
 'thank',
 'thanks',
 'ticket',
 'time',
 'today',
 'told',
 'tomorrow',
 'trying',
 'two',
 'united',
 'us',
 'usairways',
 'virginamerica',
 'wait',
 'waiting',
 'want',
 'way',
 'weather',
 'work',
 'worst',
 'would',
 

In [29]:
x_train_new = a.todense()

In [30]:
b = count_vec.transform(x_test['text'])

In [31]:
x_test_new = b.todense()

In [32]:
from sklearn.naive_bayes import MultinomialNB

In [33]:
clf = MultinomialNB()

In [34]:
clf.fit(x_train_new,y_train)
y_pred = clf.predict(x_test_new)

In [35]:
y_pred_label = []
for i in range(len(y_pred)):
    y_pred_label.append(class_names[y_pred[i]])

In [36]:
np.savetxt("Twiiter_predictions.csv",y_pred_label,fmt="%s")